In [144]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from datetime import date

## Crawling!

#### 1 : main title -> click
#### 2 : subtitle -> click
#### 3 : content -> get "Quesion" and "Main contents"
#### 4 : English content -> click
#### 5 : English content -> get "Quesion" and "Main contents"

In [53]:
driver = webdriver.Chrome()
driver.get("https://www.gotquestions.org/Korean/")

a_tags=driver.find_elements(By.TAG_NAME,"a")
print(len(a_tags))

52


In [54]:
# 영어를 포함한 값 추출해보기 - test
final=pd.DataFrame(columns=["crawling_date","big_title_kor","Question_KOR","Answer_KOR","URL_KOR","Question_ENG","Answer_ENG","URL_ENG"])

In [55]:
for i in range(1, 52): # 첫 번째 클릭 (대주제)
    

    # 1 : main title -> click
    try :
        element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{i}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
        big_title_kor = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{i}]').text # 대주제도 데이터 프레임에 넣기
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)
    

    except NoSuchElementException:
        time.sleep(0.5)
        break

    # 2 : subtitle -> click
    for ii in range(1, 120): # 두 번째 클릭 (소주제) 

        # 소주제 개수는 대주제마다 달라서, range()로 일단 1부터 120까지 범위 정해놓고
        # 반복문을 실행할 수 없는 상황, 찾으려는 요소가 없어서 더 이상 작업할 수 없을 때 반복문 종료
        try:
            element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{ii}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
            driver.execute_script("arguments[0].click();", element)
            time.sleep(0.5)
        
        # 만약 버튼이 없다면 초기 화면으로 돌아간 후 초기 반복문으로 이동
        except NoSuchElementException:
            driver.get("https://www.gotquestions.org/Korean/")
            time.sleep(0.5)
            break
        
        # 3 : content -> get "Quesion" and "Main contents"
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
            
        # crawling 하는 날짜 date 넣기
        crawling_date = date.today()
        # 제목(소주제)는 <h1></h1> 사이에 존재
        title_kor = soup.find('h1').get_text(strip = True)
        # 현재 url
        url_kor = driver.current_url
        
        # 답변 내용은 "답변"과 "English" 사이에 존재한다는 공통점
        content_str_kor = str(soup.get_text(separator="\n"))
        start_kor = content_str_kor.find('답변') + len('답변') # 답변의 시작 지점
        end_kor = content_str_kor.find('English') # 답변의 끝 지점
        answer_html_kor = content_str_kor[start_kor : end_kor]
        answer_kor = BeautifulSoup(answer_html_kor, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기

        # ----------- #

        # 4 : English content -> click
        element = driver.find_element(By.LINK_TEXT, "English")
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
            
        # 5 : English content -> get "Quesion" and "Main contents"
        # 영어 제목(소주제) <h1></h1> 사이에 존재
        title_eng = soup.find('h1').get_text(strip = True)

        # 현재 url
        url_eng = driver.current_url
            
        # 답변 내용은 "Question"과 "Return to:" 사이에 존재한다는 공통점
        content_str_eng = str(soup.get_text(separator="\n"))
        start_eng = content_str_eng.find('Answer') + len('Answer') # 답변의 시작 지점
        end_eng = content_str_eng.find('Return to:') # 답변의 끝 지점
        answer_html_eng = content_str_eng[start_eng : end_eng]
        answer_eng = BeautifulSoup(answer_html_eng, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기

        
        # 파일 저장
        new_row = pd.DataFrame({"crawling_date":[crawling_date], "big_title_kor" : [big_title_kor],
                                "Question_KOR": [title_kor], "Answer_KOR": [answer_kor], "URL_KOR": [url_kor],
                                "Question_ENG": [title_eng], "Answer_ENG": [answer_eng], "URL_ENG": [url_eng]})
        final = pd.concat([final, new_row], ignore_index=True)
        
        driver.back()
        time.sleep(0.5)

        driver.back()
        time.sleep(0.5)

In [56]:
final

,crawling_date,big_title_kor,Question_KOR,Answer_KOR,URL_KOR,Question_ENG,Answer_ENG,URL_ENG
0,2025-04-29,복음제시,영생을 얻으셨습니까? 예수 그리스도를 믿음으로 말미암아 영원한 생명을 소유하셨습니까?,"성경은 영생에 이르는 분명한 길을 제시합니다. 먼저, 우리는 하나님 앞에 죄인임을 ...",https://www.gotquestions.org/Korean/Korean-Got...,Got Eternal Life?,The Bible presents a clear path to eternal lif...,https://www.gotquestions.org/eternal-life.html
1,2025-04-29,복음제시,용서를 받으셨습니까? 하나님께 죄 사함을 받는 방법은 무엇입니까?,"‘용서하다 ’라는 단어는 과거에 저지른 잘못을 덮어주고, 눈감아주며, 빚을 청산해...",https://www.gotquestions.org/Korean/Korean-Got...,Got Forgiveness?,"Acts 13:38\n declares, ""Therefore, my brothers...",https://www.gotquestions.org/got-forgiveness.html
2,2025-04-29,복음제시,죄인의 기도란 무엇인가?,‘죄인의 기도(Sinner’s Prayer)’는 자신이 죄인이며 구주가 필요하다는 ...,https://www.gotquestions.org/Korean/Korean-Sin...,What is the sinner’s prayer?,The sinner’s prayer is a prayer a person prays...,https://www.gotquestions.org/sinners-prayer.html
3,2025-04-29,복음제시,예수님만이 천국에 이르는 유일한 길인가?,그렇습니다. 예수님만이 천국에 이르는 유일한 길입니다. 이런 배타적인 표현은 포스...,https://www.gotquestions.org/Korean/Korean-Jes...,Is Jesus the only way to heaven?,"Yes, Jesus is the only way to heaven. Such an ...",https://www.gotquestions.org/Jesus-only-way.html
4,2025-04-29,복음제시,죽은 뒤 천국에 간다는 것을 어떻게 확신할 수 있는가?,여러분은 자신이 영생을 얻었고 죽은 뒤 천국에 갈 것이라는 사실을 확신하십니까? 하...,https://www.gotquestions.org/Korean/Korean-Kno...,How can I know for sure that I will go to heav...,Do you know for certain that you have eternal ...,https://www.gotquestions.org/know-sure-Heaven....
...,...,...,...,...,...,...,...,...
1435,2025-04-29,성경에 나오는 사람들에 대한 질문,아볼로는 누구입니까?,"아볼로는 전도자이자 변증가였고, 교회 지도자이며, 사도 바울의 친구였습니다. 아볼로...",https://www.gotquestions.org/Korean/Korean-who...,Who was Apollos?,"Apollos was an evangelist, apologist, church l...",https://www.gotquestions.org/who-Apollos.html
1436,2025-04-29,성경에 나오는 사람들에 대한 질문,바나바의 삶을 통해 우리는 무엇을 배울 수 있습니까?,"사도행전에는 사도들이 바나바라고 불렀던, 요셉이라는 이름을 가진 구브로 출신의 레위...",https://www.gotquestions.org/Korean/Korean-lif...,Who was Barnabas in the Bible?,"In the book of Acts, we find a Levite from \nC...",https://www.gotquestions.org/life-Barnabas.html
1437,2025-04-29,성경에 나오는 사람들에 대한 질문,에스더의 삶을 통해 우리는 무엇을 배울 수 있습니까?,에스더는 페르시아의 왕비가 되어 유대 민족을 전멸하기 위해 온 민족을 죽이려는 음모...,https://www.gotquestions.org/Korean/Korean-lif...,Who was Esther in the Bible?,Esther is the Jewish maiden who became queen o...,https://www.gotquestions.org/life-Esther.html
1438,2025-04-29,성경에 나오는 사람들에 대한 질문,에스겔의 삶을 통해 우리는 무엇을 배울 수 있습니까?,에스겔의 이름은 '하나님께서 강하게 하심'이라는 뜻을 가지고 있습니다. 그는 예루살...,https://www.gotquestions.org/Korean/Korean-lif...,Who was Ezekiel in the Bible?,"Ezekiel, whose name means “strengthened by God...",https://www.gotquestions.org/life-Ezekiel.html


In [57]:
final.to_excel(f"/Users/haley/Desktop/2025-1/DS1/code/preprocessing/GodQuestions/GodQuestions_raw_Kor_{crawling_date}.xlsx",index=False)

## Preprocessing

#### duplicated check

In [60]:
# url 주소가 duplicated 되는 것?
final["URL_KOR_remove"]=final["URL_KOR"].str.replace(r"https://www.gotquestions.org/Korean/Korean-", "", regex=True)
final["URL_KOR_remove"]=final["URL_KOR_remove"].str.replace(r".html", "", regex=True)

final["URL_KOR_remove"]

0       Got-Eternal-Life
1        Got-Forgiveness
2         Sinners-Prayer
3             Jesus-Only
4              Know-Sure
              ...       
1435         who-Apollos
1436       life-Barnabas
1437         life-Esther
1438        life-Ezekiel
1439          life-James
Name: URL_KOR_remove, Length: 1440, dtype: object

In [74]:
# duplicated row
print(final[final.duplicated("URL_KOR_remove")].shape[0])

45


In [76]:
final[final.duplicated("URL_KOR_remove")].sort_values("URL_KOR_remove")

,crawling_date,big_title_kor,Question_KOR,Answer_KOR,URL_KOR,Question_ENG,Answer_ENG,URL_ENG,URL_KOR_remove
66,2025-04-29,하나님에 관한 질문들,하나님의 속성은 무엇인가?,"하나님의 말씀인 성경은 우리에게 하나님이 어떤 분이신지, 어떤 분이 아니신지를 알려...",https://www.gotquestions.org/Korean/Korean-Att...,What are the attributes of God?,"The Bible, God’s Word, tells us what God is li...",https://www.gotquestions.org/attributes-god.html,Attributes-God
324,2025-04-29,성경에 관한 질문들,성경은 정말 하나님의 말씀인가?,이 질문에 대한 답은 우리가 성경을 어떻게 바라볼 것이며 그것이 우리 삶에 얼마나 ...,https://www.gotquestions.org/Korean/Korean-Bib...,Is the Bible truly God’s Word?,Our answer to this question will not only dete...,https://www.gotquestions.org/bible-god-word.html,Bible-Word
999,2025-04-29,창조에 관한 질문들,가인의 아내는 누구인가? 가인은 누구와 결혼한 것인가?,성경은 가인의 아내가 누구였는지 분명하게 언급하지 않습니다. 단지 가인의 아내는 그...,https://www.gotquestions.org/Korean/Korean-Cai...,Who was Cain’s wife?,The Bible does not specifically say who Cain’s...,https://www.gotquestions.org/Cains-wife.html,Cain-wife
615,2025-04-29,신앙생활에 관한 질문들,기독교인은 구약의 율법을 지켜야 하는가?,그리스도인과 율법의 관계를 이해하기 위한 열쇠는 본래 구약성서의 율법이 지금의 그리...,https://www.gotquestions.org/Korean/Korean-Chr...,Do Christians have to obey the Old Testament law?,The key to understanding the relationship betw...,https://www.gotquestions.org/Christian-law.html,Christian-Law
135,2025-04-29,예수 그리스도에 관한 질문들,그리스도의 신성은 성경적인가?,예수께서 자신에 대해 분명한 주장을 하셨을 뿐만 아니라 그분의 제자들 또한 그리스도...,https://www.gotquestions.org/Korean/Korean-Dei...,Is the deity of Christ biblical?,In addition to Jesus’ specific claims about Hi...,https://www.gotquestions.org/deity-of-Christ.html,Deity-Christ
64,2025-04-29,하나님에 관한 질문들,하나님은 존재하는가?,하나님의 존재는 증명할 수 없으며 반증할 수 없습니다. 성경은 하나님이 존재하신다는...,https://www.gotquestions.org/Korean/Korean-Doe...,Does God exist?,Whether God exists is one of the most basic an...,https://www.gotquestions.org/Does-God-exist.html,Does-God-Exist
1135,2025-04-29,인생결정에 관한 질문들,내 삶을 향한 하나님의 뜻을 아는 방법은 무엇인가?,하나님의 뜻을 아는 것은 중요한 일입니다. 예수님은 아버지의 뜻을 알고 행하는 사람...,https://www.gotquestions.org/Korean/Korean-God...,How can I know God’s will for my life?,It is important to know God’s will. Jesus said...,https://www.gotquestions.org/know-God-will.html,God-will
614,2025-04-29,신앙생활에 관한 질문들,내 삶을 향한 하나님의 뜻을 아는 방법은 무엇인가?,하나님의 뜻을 아는 것은 중요한 일입니다. 예수님은 아버지의 뜻을 알고 행하는 사람...,https://www.gotquestions.org/Korean/Korean-God...,How can I know God’s will for my life?,It is important to know God’s will. Jesus said...,https://www.gotquestions.org/know-God-will.html,God-will
203,2025-04-29,성령에 관한 질문들,성령님은 누구신가?,우리는 성령님의 정체에 대해 많이들 오해합니다. 어떤 사람들은 성령님을 단순히 신비...,https://www.gotquestions.org/Korean/Korean-Hol...,Who is the Holy Spirit?,There are many misconceptions about the identi...,https://www.gotquestions.org/who-Holy-Spirit.html,Holy-Spirit
65,2025-04-29,하나님에 관한 질문들,하나님은 실제로 존재하는가?,우리는 하나님이 세 가지 방법을 통해 우리에게 자신을 나타내셨기 때문에 그분이 실제...,https://www.gotquestions.org/Korean/Korean-Is-...,Is God real?,We know that God is real because He has reveal...,https://www.gotquestions.org/is-God-real.html,Is-God-Real


In [ ]:
# duplicated row remove

#### ENG Answer Preprocessing 

In [ ]:
# 1
# Find : "Have you ~ button below ~""
final["Answer_ENG"].apply(lambda x: re.findall(r"\nHave you[\s\S]*?button below[\s\S]+",x))

0       [\nHave you made a decision for Christ because...
1       [\nHave you made a decision for Christ because...
2                                                      []
3       [\nHave you made a decision for Christ because...
4       [\nHave you made a decision for Christ because...
                              ...                        
1435                                                   []
1436                                                   []
1437                                                   []
1438                                                   []
1439                                                   []
Name: Answer_ENG, Length: 1440, dtype: object

In [87]:
# Remove : "Have you ~ button below ~""
final["Answer_ENG"] = final["Answer_ENG"].str.replace(r"\nHave you[\s\S]*?button below[\s\S]+", "", regex=True)

In [88]:
final["Answer_ENG"]

0       The Bible presents a clear path to eternal lif...
1       Acts 13:38\n declares, "Therefore, my brothers...
2       The sinner’s prayer is a prayer a person prays...
3       Yes, Jesus is the only way to heaven. Such an ...
4       Do you know for certain that you have eternal ...
                              ...                        
1435    Apollos was an evangelist, apologist, church l...
1436    In the book of Acts, we find a Levite from \nC...
1437    Esther is the Jewish maiden who became queen o...
1438    Ezekiel, whose name means “strengthened by God...
1439    James was a son of Mary and Joseph and therefo...
Name: Answer_ENG, Length: 1440, dtype: object

In [ ]:
# 2
# Find : "Related Articles ~"
final["Answer_ENG"].apply(lambda x: re.findall(r"\nRelated Articles[\s\S]+",x))

0                                                      []
1                                                      []
2       [\nRelated Articles\n\n\nWhat is the prayer of...
3                                                      []
4                                                      []
                              ...                        
1435    [\nRelated Articles\n\n\nWho was Timothy in th...
1436    [\nRelated Articles\n\n\nWho was Luke in the B...
1437    [\nRelated Articles\n\n\nWho was Hadassah in t...
1438    [\nRelated Articles\n\n\nWhy did God kill Ezek...
1439    [\nRelated Articles\n\n\nWhat can we learn fro...
Name: Answer_ENG, Length: 1440, dtype: object

In [92]:
# Remove : "Related Articles ~"
final["Answer_ENG"] = final["Answer_ENG"].str.replace(r"\nRelated Articles[\s\S]+", "", regex=True)

In [93]:
final["Answer_ENG"]

0       The Bible presents a clear path to eternal lif...
1       Acts 13:38\n declares, "Therefore, my brothers...
2       The sinner’s prayer is a prayer a person prays...
3       Yes, Jesus is the only way to heaven. Such an ...
4       Do you know for certain that you have eternal ...
                              ...                        
1435    Apollos was an evangelist, apologist, church l...
1436    In the book of Acts, we find a Levite from \nC...
1437    Esther is the Jewish maiden who became queen o...
1438    Ezekiel, whose name means “strengthened by God...
1439    James was a son of Mary and Joseph and therefo...
Name: Answer_ENG, Length: 1440, dtype: object

In [ ]:
# 3
# Find : "For Further Study ~"
final["Answer_ENG"].apply(lambda x: re.findall(r"\nFor Further Study[\s\S]+",x))

0                                                      []
1                                                      []
2                                                      []
3                                                      []
4                                                      []
                              ...                        
1435    [\nFor Further Study\n\n\nThe Complete Book of...
1436    [\nFor Further Study\n\n\nThe Complete Book of...
1437    [\nFor Further Study\n\n\nThe Complete Book of...
1438    [\nFor Further Study\n\n\nThe Complete Book of...
1439    [\nFor Further Study\n\n\nThe Complete Book of...
Name: Answer_ENG, Length: 1440, dtype: object

In [95]:
# Remove : "For Further Study ~"
final["Answer_ENG"] = final["Answer_ENG"].str.replace(r"\nFor Further Study[\s\S]+", "", regex=True)

In [ ]:
final["Answer_ENG"]

0       The Bible presents a clear path to eternal lif...
1       Acts 13:38\n declares, "Therefore, my brothers...
2       The sinner’s prayer is a prayer a person prays...
3       Yes, Jesus is the only way to heaven. Such an ...
4       Do you know for certain that you have eternal ...
                              ...                        
1435    Apollos was an evangelist, apologist, church l...
1436    In the book of Acts, we find a Levite from \nC...
1437    Esther is the Jewish maiden who became queen o...
1438    Ezekiel, whose name means “strengthened by God...
1439    James was a son of Mary and Joseph and therefo...
Name: Answer_ENG, Length: 1440, dtype: object

In [111]:
# 4
# Find : "\n{1,}"
final["Answer_ENG"].apply(lambda x: re.findall(r"\n{1,}",x))[150]

['\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n\n\n\n\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n\n\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n\n\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n\n\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n\n\n\n\n\n\n\n\n\n']

In [115]:
# Remove : "\n{1,}"
final["Answer_ENG"] = final["Answer_ENG"].str.replace(r"\n{1,}", "", regex=True)

In [118]:
final["Answer_ENG"][121]

'There are at least 169 verses in the Bible that refer to the ways God provides for us. Philippians 4:19 puts it simply: “My God will supply all your needs according to His riches in glory in Christ Jesus.” While prosperity seekers might always be looking for money or possessions to miraculously arrive, we should take a closer look at what God desires to provide for us.Like any good parent, God would never give us what He knows would harm us. His intent is to help us develop Christlikeness so that we become salt and light in the world (Matthew 5:13–14). God does not want us to see Him as a heavenly source of mere material possessions. Acquiring things is not the fundamental goal of this life (Luke 12:15).God differentiates between our needs and our wants because He knows that where our treasure is our heart is also (Matthew 6:21). He wants us to know that this world is not our home and that part of what we need is to shift our focus to the eternal life while still living this one.God i

In [ ]:
# 5
#...? anything else..?

In [ ]:
# streamlit에서 바로 결과 확인하기
# https://isaac-christian.tistory.com/entry/Streamlit-%EA%B0%9C%EB%B0%9C%EC%9E%90-%ED%99%98%EA%B2%BD-%EA%B5%AC%EC%B6%95-VS-Code%EC%9D%98-%EA%B0%80%EC%83%81-%ED%99%98%EA%B2%BD%EC%97%90-Streamlit-%EC%84%A4%EC%B9%98%ED%95%98%EA%B8%B0

#### Korean Anwer Preprocessing

In [122]:
final["Answer_KOR"][1]

'‘용서하다 ’라는 단어는 과거에 저지른 잘못을  덮어주고, 눈감아주며, 빚을 청산해준다는 뜻입니다. 우리는 누군가에게 잘못을 저지르면   그 사람과의 관계를 회복하기 위해  용서를 구합니다. 그런데 용서는 잘못한 사람이 용서받을 만한 사람이기 때문에 허락되는 것이 아닙니다. 용서받아 마땅한 사람은 아무도 없습니다. 용서란 사랑, 자비, 그리고 은혜의 행위입니다. 용서는 누군가 나에게 어떤 잘못을 했더라도, 그 사람에게 원한을 품지 않기로  결정하는 것입니다. \n\n성경은 우리 모두가 하나님께 용서받을 필요가 있다고 말합니다. 우리는 모두 죄를 범했습니다. 전도서 7장 20절은 “선을 행하고 전혀 죄를 범하지 아니하는 의인은 세상에 없기 때문이로다”라고 선언합니다. 요한일서 1장 8절은 “만일 우리가 죄가 없다고 말하면 스스로 속이고 또 진리가 우리 속에 있지 아니할 것이요”라고 말합니다. 모든 죄는 궁극적으로 하나님을 대적하는 행위입니다(시 51: 4). 결과적으로, 우리 모두는 하나님의 용서가 절실히 필요합니다. 만일 우리가 죄 사함을 받지 못한다면, 우리는 죄의 결과로 영원히 고통당하게 될 것입니다(마 25: 46; 요 3: 36). \n\n\n용서 – 어떻게 받을 수 있습니까?\n\n감사하게도, 하나님은 사랑과 자비가 풍성한 분이십니다. 또한 하나님은 우리를 용서해주시길 원하십니다! 베드로후서 3장 9절은 “...오직 주께서는 너희를 대하여 오래 참으사 아무도 멸망하지 아니하고 다 회개하기에 이르기를 원하시느니라”고 말합니다. 하나님은 우리를 용서해주시길 바라십니다. 그래서 우리가 용서받을 수 있는 길을 친히 마련해 주셨습니다. \n\n우리의 죄에 대한 형벌은 사망뿐입니다. 로마서 6장 23절 상반절은 “죄의 삯은 사망이요…”라고 선언합니다. 영원한 죽음은 우리가 지은 죄로 인해 얻은 것입니다. 하나님은 그분의 완전한 계획 가운데 인간이 되셨는데, 그가 바로 예수 그리스도이십니다(요 1: 1, 14). 예수님은 우리가 마땅히 받아야 할 ‘죽음’이라는 

In [132]:
# 1
# Find : "이곳의 글을 읽고 ~"
final["Answer_KOR"].apply(lambda x: re.findall(r"\n이곳의 글을 읽고[\s\S]+",x))

0       [\n이곳의 글을 읽고 예수 그리스도를 믿기로 마음의 결정을 내리셨습니까? 그렇다면...
1       [\n이곳의 글을 읽고 예수 그리스도를 믿기로 마음의 결정을 내리셨습니까? 그렇다면...
2       [\n이곳의 글을 읽고 예수 그리스도를 믿기로 마음의 결정을 내리셨습니까? 그렇다면...
3       [\n이곳의 글을 읽고 예수 그리스도를 믿기로 마음의 결정을 내리셨습니까? 그렇다면...
4       [\n이곳의 글을 읽고 예수 그리스도를 믿기로 마음의 결정을 내리셨습니까? 그렇다면...
                              ...                        
1435                                                   []
1436                                                   []
1437                                                   []
1438                                                   []
1439                                                   []
Name: Answer_KOR, Length: 1440, dtype: object

In [135]:
# Remove : "이곳의 글을 읽고 ~"
final["Answer_KOR"]=final["Answer_KOR"].str.replace(r"\n이곳의 글을 읽고[\s\S]+", "", regex=True)

In [137]:
final["Answer_KOR"]

0       성경은 영생에 이르는 분명한 길을 제시합니다. 먼저, 우리는 하나님 앞에 죄인임을 ...
1       ‘용서하다 ’라는 단어는 과거에 저지른 잘못을  덮어주고, 눈감아주며, 빚을 청산해...
2       ‘죄인의 기도(Sinner’s Prayer)’는 자신이 죄인이며 구주가 필요하다는 ...
3       그렇습니다. 예수님만이 천국에 이르는  유일한 길입니다. 이런 배타적인 표현은 포스...
4       여러분은 자신이 영생을 얻었고 죽은 뒤 천국에 갈 것이라는 사실을 확신하십니까? 하...
                              ...                        
1435    아볼로는 전도자이자 변증가였고, 교회 지도자이며, 사도 바울의 친구였습니다. 아볼로...
1436    사도행전에는 사도들이 바나바라고 불렀던, 요셉이라는 이름을 가진 구브로 출신의 레위...
1437    에스더는 페르시아의 왕비가 되어 유대 민족을 전멸하기 위해 온 민족을 죽이려는 음모...
1438    에스겔의 이름은 '하나님께서 강하게 하심'이라는 뜻을 가지고 있습니다. 그는 예루살...
1439    야고보는 마리아와 요셉의 아들이었기 때문에, 예수님의 이복형제인 요셉, 시몬, 유다...
Name: Answer_KOR, Length: 1440, dtype: object

In [141]:
# 2
# Find : "\n{1,}"
final["Answer_KOR"].apply(lambda x: re.findall(r"\n{1,}",x))[172]

['\n\n',
 '\n\n',
 '\n\n',
 '\n\n',
 '\n\n',
 '\n\n',
 '\n\n',
 '\n\n',
 '\n\n',
 '\n\n',
 '\n\n']

In [142]:
# Remove : "\n{1,}"
final["Answer_KOR"] = final["Answer_KOR"].str.replace(r"\n{1,}", "", regex=True)

In [143]:
final["Answer_KOR"][172]

'타나크 (Tanakh, 히브리어 성경/구약성경) 속에 들어있는 모든 메시아 관련 예언 중에서 유대교 메시아의 도래와 관련한 가장 위대한 예언은 아마도 이사야서 53장에서 찾을 수 있을 것입니다. “고난 당하는 종”으로 잘 알려진 예언서의 이 부분은 오랫동안 유대교의 역사적 랍비들에 의해 언젠가 시온에 오실 구원자를 이야기는 것으로 해석되어 왔습니다. 여기 제시된 예들을 통해 이사야 53장의 “고난 당하는 종”의 신분에 대해 유대교가 전통적으로 믿어왔던 것들을 살펴 볼 수 있습니다. 바벨로니아 탈무드 (Babylonian Talmud )는 다음과 같이 말합니다. “메시아, 그의 이름은 무엇인가? 랍비가 대답하길, 레퍼 스콜라(Leper Scholar)라 한다.  ‘그는 명백히 우리의 고통을 짊어지고 우리의 슬픔을 감당하나, 우리는 그를 나병 환자, 하나님의 벌을 받는 자, 고통 당하는 자로 여긴다…”고 했기 때문이다” (산헤드린 98). 미드라쉬 루스 라바(Midrash Ruth Rabbah)는 이렇게 말합니다. “(루스 2:14의) 또 다른 설명 : 그는 메시아 임금에 대하여 이야기하고 있다; ‘여기로 오라’, 보좌로 가까이 이끈다; ‘그리고 떡을 먹으라’, 즉 왕국의 떡을 의미; ‘그리고 너의 작은 조각을 초에 담그라’, 이는 ‘그러나 그는 우리의 죄악을 위해 상처를 입었고 우리의 죄로 인해 멍이 들었다’고 했듯이 그의 징벌을 의미한다.” 타르굼 조나단 (Targum Jonathan)은 “나의 종 메시아가 번영하는 것을 보아라; 그는 높아지고 커지며 극도로 강해질 것이다”라고 말합니다. 조하르 (Zohar)는 다음과 같이 말합니다. “ ‘그는 우리의 죄악 때문에 상처를 입었다’ 등…에덴 동산에는 병든 자들의 궁전이라 불리는 궁전이 있다; 이 궁전에 메시아가 들어가서, 이스라엘의 모든 병들, 모든 고통들, 모든 죄악들을 불러내신다 ; 그것들이 모두 그에게로 와서 안식한다. 그가 이스라엘에게서 그것들을 덜어 자기가 직접 짊어지지 않았다면 율법을 어김으로 말미암

In [ ]:
# 3
#...? anything else..?